In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# CIFAR-10

## VGG-16

In [9]:
from torchvision.datasets import CIFAR10
from torchvision import transforms

train_transforms = transforms.Compose([transforms.ToTensor(),
                                transforms.RandomResizedCrop(size=(224, 224), scale=(0.9, 1.0)),
                                transforms.RandomHorizontalFlip()])
test_trainsorms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224))])

train_dataset = CIFAR10(root='../dataset/cifar10',
                        train=True,
                        transform=train_transforms,
                        download=False)
test_dataset = CIFAR10(root='../dataset/cifar10',
                       train=False,
                       transform=test_trainsorms,
                       download=False)

In [10]:
from torch.utils.data import DataLoader

batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [11]:
from model.vgg import VGG16, VGG19
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
from torch.nn import CrossEntropyLoss
from evaluation.metrics import Metric
from evaluation.eval import ClassificationEvaluator

model = VGG16(10)
optimizer = Adam(model.parameters(), lr=1e-2, betas=(0.9, 0.98), eps=1e-9, weight_decay=5e-4)
scheduler = ExponentialLR(optimizer, gamma=0.9)
loss_fn = CrossEntropyLoss()
metrics = [Metric.Accuracy, ]
evaluator = ClassificationEvaluator(metrics)

In [12]:
from trainer.supervised import SimpleTrainer

trainer = SimpleTrainer(model, train_loader, test_loader, loss_fn=loss_fn, optimizer=optimizer, evaluator=evaluator, eval_freq=3000)

In [ ]:
epochs = 10

for epoch in range(epochs):
    trainer.train()
    scheduler.step()